In [14]:
import pandas as pd
from pyxlsb import open_workbook
import io
import csv

In [67]:
filename = r'\\corp.tele2.ru\plm_cluster\All\_Плановый Sales mix\Расчет целевого Sales mix_июнь.xlsb'
with open_workbook(filename) as wb:
    output = io.StringIO()
    writer = csv.writer(output, quoting=csv.QUOTE_NONNUMERIC)
    with wb.get_sheet('Channels_MIX') as sheet:
        for row in sheet.rows():
            csv_line = [r.v for r in row]
            writer.writerow(csv_line)
    output.seek(0)

In [68]:
sm = pd.read_csv(output, skiprows = 2)
sm.head(2)

,Регион,Channels,МР,Кластер,ТП в продажах,GI Total,Классический,Мой Tele2,Мой разговор,Мой онлайн,...,Unnamed: 100,Классический.10,Мой Tele2.10,Мой разговор.10,Мой онлайн.10,Мой онлайн +.10,Безлимит.10,Везде онлайн.10,Other.10,Unnamed: 109
0,Владимир,Total,Центр,Challenger Elite,Мой онлайн +,9619.098161,1426.810211,1299.699987,2289.242859,2242.048240,...,0.000198,-0.2584858224992843,-0.25905977575374695,0.1768083010176118,0.2872158359237269,0.09794259537561567,-0.19611042360499165,0x7,-0.37289704835420767,NaN
1,Владимир,MB,Центр,Challenger Elite,Мой онлайн +,4512.662879,409.949398,214.633193,1037.751487,1253.457846,...,NaN,-0.3267461760664291,-0.39431549100468977,0.14381912684342257,0.2652998304354086,0.08951532979038057,-0.23403602048887817,0x7,-0.38035384342619116,NaN


In [110]:
sm.iloc[:,[0,1,2,3,72,73,74,75,76,77,78,79]].head(2)

,Регион,Channels,МР,Кластер,Классический.7,Мой Tele2.7,Мой разговор.7,Мой онлайн.7,Мой онлайн +.7,Безлимит.7,Везде онлайн.7,Other.7
0,Владимир,Total,Центр,Challenger Elite,1058.0,963.0,2694.0,2886.0,961.0,578.0,0.0,481.0
1,Владимир,MB,Центр,Challenger Elite,276.0,130.0,1187.0,1586.0,684.0,254.0,0.0,395.0


In [111]:
sm1 = sm.iloc[:,[0,1,2,3,72,73,74,75,76,77,78,79]]
sm1.columns = ['region','channel','macroregion','cluster', 'Классический', 'Мой Tele2','Мой разговор',
                'Мой онлайн','Мой онлайн +','Безлимит','Везде онлайн','Other']
sm1.head(2)

,region,channel,macroregion,cluster,Классический,Мой Tele2,Мой разговор,Мой онлайн,Мой онлайн +,Безлимит,Везде онлайн,Other
0,Владимир,Total,Центр,Challenger Elite,1058.0,963.0,2694.0,2886.0,961.0,578.0,0.0,481.0
1,Владимир,MB,Центр,Challenger Elite,276.0,130.0,1187.0,1586.0,684.0,254.0,0.0,395.0


In [114]:
sm2 = sm1.melt(id_vars = ['region','channel','macroregion','cluster'],
         value_vars=['Классический','Мой Tele2','Мой разговор','Мой онлайн',
                     'Мой онлайн +','Безлимит','Везде онлайн','Other'],
         var_name='tariff',
         value_name='subs_count')
sm2 = sm2.loc[(sm2.channel != 'Total'),:].dropna()
sm2.head()

,region,channel,macroregion,cluster,tariff,subs_count
1,Владимир,MB,Центр,Challenger Elite,Классический,276.0
2,Владимир,LA,Центр,Challenger Elite,Классический,274.0
3,Владимир,LS,Центр,Challenger Elite,Классический,55.0
4,Владимир,Stands,Центр,Challenger Elite,Классический,15.0
5,Владимир,FA,Центр,Challenger Elite,Классический,423.0


In [115]:
sm2pivot = sm2.groupby(['tariff','channel']).sum().reset_index()
sm2pivot.pivot(index='tariff', columns='channel', values='subs_count')

channel,FA,FS,LA,LS,MB,Other,Stands
tariff,,,,,,,
Other,558.0,193.0,1324.0,927.0,18263.0,886.0,279.0
Безлимит,743.0,116.0,6379.0,2572.0,17992.0,628.0,3624.0
Везде онлайн,170.0,35.0,6552.0,1522.0,54785.0,2110.0,452.0
Классический,15414.0,1400.0,27486.0,9967.0,30510.0,1432.0,3571.0
Мой Tele2,9968.0,2696.0,37358.0,6125.0,11648.0,690.0,1586.0
Мой онлайн,27916.0,5384.0,81346.0,20526.0,124577.0,4357.0,11594.0
Мой онлайн +,1921.0,1523.0,33082.0,6615.0,62079.0,1960.0,2072.0
Мой разговор,13351.0,3476.0,55281.0,17880.0,98013.0,3311.0,3908.0


In [116]:
newfilename = r'C:\Users\igor.i.plotnikov\Documents\My Tableau Repository\Datasources\TheGreatDashboard\Sales Mix.xlsx'
sm2.to_excel(newfilename, index = False)